# Designing a Self-organizing Endpoint with Even Flow

This is an example notebook showing how to build complex systems from simple components using [evenflow](https://github.com/willkurt/evenflow/), a library which seeks to create a framefork for *self-organizing* componsable functions. The idea here is to show some of the power of this type of thinking, which becomes increasingly important as we start to develop more advanced agent-based LLM programs. This code doesn't use any LLMs, but it's not hard to see how some of the patterns here could be hugely beneficial for programs that did.

In this example we'll be building out of a mock up of a json endpoint that takes in samples and returns the standard error of those samples using Self-organizing functions built with `evenflow`.

Let's start by importing the two necessary `evenflow` functions: `flowable` (a decorator) and `compose_flow` which assembles the `flowable` functions into a larger function.

In [1]:
from evenflow import flowable, compose_flow
from evenflow.compose import  _compose_flow, build_execution_stages
from evenflow.base import flow_sub
import numpy as np

## Calculating Standard Error


Standard Error (se) is our uncertainity in the estimate of the mean of population based on samples we've observed. Mathematically it's faily simple to express. Given that

- $\sigma^2$ is the variance our population (thus $\sigma$ is the *standard deviation*)
- $N$ is the number of observations in our sample

Then Standard Error is defined as:

$$\text{se}=\frac{\sigma}{\sqrt{N}}$$

While this is simple to represent mathematically, I've always found that *calculating* it is a bit tedious. 

The typical presentation of how to calcuate *standard deviation* online is as follows (this procedure is from [Khan Academy](https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/variance-standard-deviation-population/a/calculating-standard-deviation-step-by-step)):

1. Find the mean.
2. For each data point, find the square of its distance to the mean.
3. Sum the values from Step 2.
4. Divide by the number of data points.
5. Take the square root.

Typically we would implement this as a *single* function in Python. But to show off the power of `evenflow` we'll implment *each step* as it's own, `flowable`, function. The we'll discuss and show off some of the benefits of this approach. When using the flowable decorate we have to follow 3 easy rules:

- all functions much return values.
- the names of arguments must have consistent meaning in the system.
- use the `flowable` decorator to specify the name of the return type.

Let's look at the code!

In [2]:
# Step 1
@flowable('sample_mean')
def calc_mean(samples):
    return np.mean(samples)

# Step 2
@flowable('square_distance')
def calc_squares_distance(sample_mean, samples):
    return (samples-sample_mean)**2

# Step 3
@flowable('sum_of_squares')
def calc_sum(square_distance):
    return np.sum(square_distance)

# Step 4a - we'll need to get the n_samples (this will be useful in a bit)
@flowable('n_samples')
def calc_n_samples(samples):
    return len(samples)

# Step 4b - This is just the calculation of the variance
@flowable('var')
def variance(sum_of_squares, n_samples):
        return sum_of_squares/n_samples
    
# Step 5 - This is just calcuating the standard deviation from the variance
@flowable('std_dev')
def standard_deviation(var):
    return np.sqrt(var)

And of course we want to tie this all together into a calcuation of standard error:

In [3]:
@flowable('se')
def standard_error(std_dev, n_samples):
    return std_dev/np.sqrt(n_samples)

## What are the benefits of all this?

It's worth taking a moment to discuss this example. It may seem *excessively* contrived but even in this case there are a couple of benefits of taking the `evenflow` approach.

First is that, having implemented this function quite a few times in my life, it is easier to just build out each step one-by-one based on the instructions without having to think at all about tying the steps together (not a huge win here, but in more sophisticated functions it would definitely help).

This is *very* relevent in the world of AI and LLMs because it *might* not always be a human writing the code! I'm fairly certain an LLM can write a good function for calculating standard error, but my experience has been that as complexity increases LLMs have a harder time getting it right. When tasks are broken down into very small pieces we can be increasingly sure the LLM can acheive the goal. Don't want to write all of these flowable functions for an AI to fill out? Have the *AI* do the planning!

This brings us to a second important point. Code this grainular allows for incredible levels of testing. If an AI agent wrote the body of each of these it would not be hard to generate tests cases to verify the correctness of the code. It's also quite reasonable that an *LLM could write tests* for all of these. We start to chart the pathway towards automated, intelligent systems that are able to verify their own correctness for arbitrarily complex processes.

Finally the incredible composible nature of this code makes is *very easy* to change behavior of a complex system by simply swaping out a *single part*. For example, we could trivially change our computation of `n_samples` to use `Array.shape[0]` instead of `len` without having to even *look* at the other parts of the machinary. We'll explore a slightly more interesting case of this in a bit.


## Putting it together

We'll make this just a hair more realistic and make an imaginary endpoint that will ultimately take in a list of samples and returns a JSON response of the form:

```javascript
{
    'standard_error': <val>
}
```

Since we're in a notebook we won't actually worry about the serving part, we'll just make a simple function that takes in `se` and returns json.

In [4]:
@flowable('json_resp')
def mock_endpoint(se):
    import json
    return json.dumps({
        'standard_error': se
    })

It's worth pointing out that, for all intents and purposes, `flowable` functions are *just regular functions*. We can now easily test just the endpoint without worrying about the rest:

In [5]:
mock_endpoint(0.38)

'{"standard_error": 0.38}'

This function just takes in *standard error* and returns JSON, but recall we want an endpoint that takes in *samples* and returns JSON. This will be achieved by composing all these functions together.

In [6]:
endpoint_components = [
    mock_endpoint, calc_mean, calc_squares_distance, standard_error, 
    calc_sum, calc_n_samples, variance, standard_deviation
]

Remember: the order of the functions is irrelevant for `evenflow`, you don't have to think about it all. All we need to do is pass these to `compose_flow` and we'll get our endpoint:

In [7]:
endpoint_v1 = compose_flow(endpoint_components)
endpoint_v1

<function evenflow.compose.compute_json_resp(samples)>

As you can see, the function signature is exactly what we wanted, it takes in `samples` and returns out `json_resp`. Let's try it out!

In [8]:
endpoint_v1([1,2,3,4])

'{"standard_error": 0.5590169943749475}'

And there we have it, a self-organized endpoint that returns a JSON response giving the standard error of a collection of samples!

But what happens if we need to change something?

## Updating Systems: Adding Bessel's Correction

We've launched our endpoint and are happily serving up `standard_error` calculations for our users, but there's a problem! Some of our more statistically savy users have complained that we're not correctly calculating standard error for a *sample*. They argue we should be using [Bessel's Correction](https://en.wikipedia.org/wiki/Bessel%27s_correction) to create an unbiased estimate of the sample variance. We *could* start a complex debate about why an *unbiased* estimator is prefered since that should imply increased variance in our estimates... but the customer is always right!

With `evenflow` is very *easy* to do. First let's make a new `flowable` function that behaves like our user wants:

In [9]:
@flowable('var')
def bessels_correction(sum_of_squares, n_samples):
        return sum_of_squares/(n_samples - 1)

Notice that all that matters is that this new function returns the same *output* as our old. We could easily add more inputs if we wanted (and of course add additional `flowable` functions that output those inputs if necessary). The system is *very* robust to changing and adding inputs.

Next we remove the `variance` component, and add in our `bessels_correction` component. It's not too hard to write this from scratch, but `evenflow` has a helper function `flow_sub` which will swap out a particular component from a list with one matching its' output type:

In [10]:
endpoint_components_updated = flow_sub(endpoint_components, bessels_correction)

In [11]:
for comp in endpoint_components:
    print(comp.output)

json_resp
sample_mean
square_distance
se
sum_of_squares
n_samples
var
std_dev


Here we can see "order doesn't matter" is not just a clever trick, it allow us to very easily mutate the components of a function without worry. Let's build a new version of our endpoint:

In [12]:
endpoint_v2 = compose_flow(endpoint_components_updated)
endpoint_v2([1,2,3,4])

'{"standard_error": 0.6454972243679028}'

And there we go! Without needing knowledge or even *access* to *any* of the components, even the one we're replacing, we can trivially update the behavior of our endpoint.

## Conclusion

By treating our problem as a small number of easily composable, self-organizing components with `evenflow` we are able to create a reasonable complex system that is:

- made of very simple functions
- extremely easy to test
- easy to extend
- easy to modify *with no access to the original component code*

That last bullet point is particularly important as systems grow in complexity. You've likely worked with a large production system and had to change the behavior of a specific part. This typically involves a deep exploration of the code base and being careful not to change the behavior of the system by accident (i.e. introducing more bugs!). In a system built with `evenflow` the *only information you need* is the names of the inputs and outputs and the ability to modify *which* components are used. 

Also notice that because we worked in a functional style, we never had to change the behavior of `endpoint_v1`, in fact it still works *exactly* as it did before. This type of change would be impossible if we needed to modify the `variance` function itself.

Most of the ideas that influence `evenflow` come out of the spirit of `Additive Programming` explored in the book [Software Design For Flexiblity](https://mitpress.mit.edu/9780262045490/software-design-for-flexibility/)